In [ ]:
import numpy as np
from numpy import array as arr

import pandas
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl
# mpl.use("pdf")
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
mpl.rcParams["font.serif"] = "CMU Serif"
mpl.rcParams["font.cursive"] = "CMU Classical Serif"
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = "CMU Serif"
mpl.rcParams["mathtext.it"] = "CMU Serif:italic"
mpl.rcParams["mathtext.bf"] = "CMU Serif:bold"
mpl.rcParams["font.family"] = "serif"

# import pylandau
from scipy.optimize import curve_fit
import scipy
import uncertainties as unc
from uncertainties import unumpy as unp
from uncertainties.unumpy import uarray as uarr
from uncertainties.unumpy import nominal_values as val
from uncertainties.unumpy import std_devs as dev
from uncertainties import ufloat as uf
# import ROOT

# import my plotting stuff
import sys
sys.path.append('./PythonHelpers/')
# import PlotLib.Stats as Stats
import PlotLib.Hist1D as Hist1D
import PlotLib.Hist2D as Hist2D
import PlotLib.Plotting as Plot
from CSVimporter.importer import load_run
from CSVimporter.importer import dict_to_arr

import time

def fitfunc_gauss(x, A, mu, sigma):
    return A * np.exp(-0.5*((x-mu)/sigma)**2)

import ROOT

def get_mask_seedPix(data):
    return data[:,:,M["Charge"]] == arr([np.max(data[:,:,M["Charge"]], axis=1) for i in range(4)]).T

In [ ]:
nEvents = 4
# nEvents = 4*200
runIDs = arr([184,190,192])
runIDs = arr([190])
filepaths = {
    184:f"/home/jona/DESY/analysis_TB/output/csv/184_thr200.csv",
    190:f"/home/jona/DESY/analysis_TB/output/csv/190_thr200.csv",
    192:f"/home/jona/DESY/analysis_TB/output/csv/190_thr520.csv",
}

runs = {}
for runID in runIDs:
    runs[runID] = load_run(runID, nEvents*4, filepaths[runID])
    # runs[runID]["data"] has the following structure:
    # [ event , pixel, entry ] where entry is the index given by the map M
    
    runs[runID]["nEvts"] = min(runs[runID]["nEvts"],nEvents)
    runs[runID]["threshold"] = 200 if runID in [184,190] else 520
    runs[runID]["MTP"] = 5
    
M = runs[runIDs[0]]["M"]

def findMax(func, par, x0=10):
    def f(x, par):
        return -1 * func(x, par)
    f2 = np.vectorize(f, excluded=[1])
    return scipy.optimize.fmin(f2, x0, args=(par,), disp=False)[0]

print(M)

In [ ]:
runIDs = arr([190])
thrs = {184:200,190:200,192:550}
FilePaths = {190:"/home/jona/DESY/analysis_TB/output/190_thr200.root"}

def Import_TProfile(ObjName, rDir, rebin=30000, scaleX=1/1E9/60/60, scaleY=1000):
    TProf = rDir.Get(ObjName)    
    if not TProf:
        raise KeyError("TProfile not found: "+ObjName)        
    
    if rebin is not None:
        TProf.Rebin(30000)
        print(f"BinWidth after rebinning = {TProf.GetBinWidth(1)/1E9/60:.1f} min")
    
    binN = TProf.GetNbinsX()
    resX_t = np.zeros(binN)
    resX_val = uarr(np.zeros(binN),np.zeros(binN))
    
    for i in range(binN):
        resX_t[i] = TProf.GetBinLowEdge(i+1) + TProf.GetBinWidth(i+1)/2
        resX_val[i] = uf(TProf.GetBinContent(i+1), TProf.GetBinError(i+1))
        
    resX_t = resX_t[resX_val!=0.]
    resX_val = resX_val[resX_val!=0.]
    
    resX_t = resX_t * scaleX # convert to h
    resX_val = resX_val * scaleY # convert to um
    
    return resX_t, resX_val

for runID in runIDs:
    thr = thrs[runID]

    rFile = ROOT.TFile(FilePaths[runID])
    if not rFile:
        print("File not found at " + FilePaths[runID])
        rFile.Close()
        continue
        
    DirName = "AnalysisDUT/DSO9254A_0/local_residuals"
    rDir = rFile.Get(DirName)
    if not rDir:
        print("Dir not found: "+DirName)
        rFile.CLose()
        continue
    
    ObjName = "residualsXVsEventTime"
    resX_t, resX_val = Import_TProfile(ObjName, rDir, rebin=30000, scaleX=1/1E9/60/60, scaleY=1000)
    
    ObjName = "residualsYVsEventTime"
    resY_t, resY_val = Import_TProfile(ObjName, rDir, rebin=30000, scaleX=1/1E9/60/60, scaleY=1000)
    
    rFile.Close() 

In [ ]:
FileName = "plots/ResidualDUT-time"

fig, axs = Plot.create_fig(2,1, [5,2], sharex=True, sharey=True)

axs[0].axhline(0, color="black", lw=1, ls="--")
axs[0].errorbar(resX_t, val(resX_val), dev(resX_val), fmt=".", color="C0", lw=1, capsize=2)
text_str = f"Mean = {val(resX_val).mean():.3f} ns\n"+f"Std dev. = {val(resX_val).std():.2f} ns"
axs[0].text(0.95, 0.95, text_str, transform=axs[0].transAxes, ha="right", fontdict={"color":"black"}, va="top")

axs[1].axhline(0, color="black", lw=1, ls="--")
axs[1].errorbar(resY_t, val(resY_val), dev(resY_val), fmt=".", color="C1", lw=1, capsize=2)
text_str = f"Mean = {val(resY_val).mean():.3f} ns\n"+f"Std dev. = {val(resY_val).std():.2f} ns"
axs[1].text(0.95, 0.95, text_str, transform=axs[1].transAxes, ha="right", fontdict={"color":"black"}, va="top")

Plot.finalize(runs[runID], fig, axs, 
    title="DUT Track Residuals over Time",
    xlabel=r"time [hours]",
    ylabel= f"track intercept - DUT cluster pos.\n [μm, avg. over 33 min per entry]",
    ER1=False, param_dict={"campaign":True, "sample":True, "ER1Param":True, "recoParam":False, "fontsize":6, "thr":runs[runID]["threshold"]},
    grid=False, legend_loc="upper right")

axs[0].set_ylim(axs[0].get_ylim()[0], axs[0].get_ylim()[1]*1.35)
# axs[1].set_xlabel(r"track $y$ - DUT cluster $y$ [μm]", loc="right")

Plot.savefig(fig, FileName)